In [1]:
import pandas as pd
import re
import nltk
import csv
from nltk.corpus import stopwords

# Pre-processing needed to be done locally as the datasets were too large for Colab
filename_tracks = 'E:/Downloads/archive-features/tracks.csv'
filename_lyr = "E:/Downloads/archive-lyrics/lyrics-data.csv"
filename_gen = "E:/Downloads/archive-lyrics/artists-data.csv"

In [2]:
# Function to clean and format artist names consistently
def clean_artist_name(artist_name):
    try:
        artist_name = re.sub(r'[^\w\s]', '', artist_name)  # Remove non-alphanumeric characters
        artist_name = re.sub(r'\s+', ' ', artist_name)  # Replace multiple spaces with a single space
        artist_name = artist_name.strip() # Remove surrounding whitespace
        artist_name = artist_name.lower()
    except Exception as e:
        print(f"Error cleaning artist name: {artist_name}")
        raise e
    return artist_name

# Cleanup genre column
def split_genres(genre_string):
    if isinstance(genre_string, str):
        delimiters = [',', ';', '/']  # Hardcoded delimiters
        for delimiter in delimiters:
            genre_string = genre_string.replace(delimiter, '; ')
        genres = genre_string.split('; ')
        return genres
    else:
        return []

# Pre-process lyrics
def preprocess_text(text, stop_filter=True, flg_stemm=False, flg_lemm=True):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    words = text.split()

    ## remove Stopwords
    if stop_filter:
        stop_word_set = set(stopwords.words("english"))
        words = [word for word in words if word not in stop_word_set]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        words = [ps.stem(word) for word in words]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        words = [lem.lemmatize(word) for word in words]
            
    ## back to string from list
    text = " ".join(words)
    return text

In [3]:
# Get song dataset with musical features
df = pd.read_csv(filename_tracks)

df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [4]:
# Get lyrics dataset, skip problematic row
df_lyr = pd.read_csv(filename_lyr, skiprows=range(177701, 177702))

df_lyr.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [5]:
# Clean and format the 'ALink' column in df_lyr (already in consistent format so no need to run through clean_artist_name)
df_lyr['ALink'] = df_lyr['ALink'].str.strip('/')
df_lyr['ALink'] = df_lyr['ALink'].str.replace('-', ' ')
df_lyr['ALink'] = df_lyr['ALink'].str.lower()

# Apply the clean_artist_name function to 'artists' column in df
df['artists'] = df['artists'].apply(clean_artist_name)

# Merge the dataframes based on the matching values in 'SName', 'name', 'ALink', and 'artists' columns
df_comb = pd.merge(df, df_lyr, left_on=['name', 'artists'], right_on=['SName', 'ALink'], how='inner')

# Drop the unnecessary columns ('SName' and 'ALink')
df_comb.drop(['SName', 'ALink'], axis=1, inplace=True)

df_comb.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,SLink,Lyric,language
0,7lRFR5GJCxK87ZbVMtQSeS,Ain't Misbehavin',28,192600,0,louis armstrong,['19eLuQmk9aCobbVDHc6eek'],1923,0.665,0.254,...,0.108,0.976,0.00217,0.644,0.503,125.734,4,/louis-armstrong/aint-misbehavin.html,No one to talk to ....all by myself\nNo one to...,en
1,5JIjquTLgifuSIk0SLdo0E,Ain't Misbehavin',16,239000,0,louis armstrong,['19eLuQmk9aCobbVDHc6eek'],1955,0.517,0.684,...,0.113,0.662,0.06260,0.074,0.801,88.064,4,/louis-armstrong/aint-misbehavin.html,No one to talk to ....all by myself\nNo one to...,en
2,1DrFqUyvnPA2Y6yLrCJavg,Ain't Misbehavin',2,239000,0,louis armstrong,['19eLuQmk9aCobbVDHc6eek'],1955,0.517,0.684,...,0.113,0.662,0.06260,0.074,0.801,88.064,4,/louis-armstrong/aint-misbehavin.html,No one to talk to ....all by myself\nNo one to...,en
3,6qaVtk8MKGgKwWFtciFll6,Ain't Misbehavin',4,276240,0,louis armstrong,['19eLuQmk9aCobbVDHc6eek'],1955,0.656,0.418,...,0.179,0.624,0.03860,0.216,0.776,76.176,4,/louis-armstrong/aint-misbehavin.html,No one to talk to ....all by myself\nNo one to...,en
4,5WlMyDvyQedom0kQ3MCTow,Lazy River,14,182707,0,louis armstrong,['19eLuQmk9aCobbVDHc6eek'],1923,0.571,0.269,...,0.067,0.960,0.01290,0.121,0.490,101.845,4,/louis-armstrong/lazy-river.html,Up a lazy river by the old mill stream\nThat l...,en


In [6]:
# Get genre dataset
df_gen = pd.read_csv(filename_gen)

df_gen.head()

,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/


In [7]:
# Remove problematic NaN rows
df_gen.dropna(subset=['Artist'], inplace=True)

# Remove unnecessary columns 
df_gen.drop(['Popularity', 'Link', 'Songs'], axis=1, inplace=True)

# Apply the clean_artist_name function to 'Artist' column in df for comparison
df_gen['Artist'] = df_gen['Artist'].apply(clean_artist_name)

In [8]:
# Search and append genres from df_gen to df_comb based on artist 
for index, row in df_comb.iterrows():
    artist = row['artists']
    genres = df_gen.loc[df_gen['Artist'] == artist, 'Genres'].values
    if len(genres) > 0:
        df_comb.at[index, 'Genres'] = genres[0]

# Confirm all desired columns are present
print(df_comb.columns)

# Size check
print(len(df_comb))

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'SLink', 'Lyric',
       'language', 'Genres'],
      dtype='object')
33163


In [9]:
# Remove problematic NaN rows
df_comb.dropna(subset=['Genres'], inplace=True)

# Size check
print(len(df_comb))

32606


In [10]:
# Clean-up and split genres into list of strings
df_comb['Genres'] = df_comb['Genres'].apply(split_genres)

# Clean-up genre substrings
df_comb['Genres'] = df_comb['Genres'].apply(lambda x: [genre.strip() for genre in x])

In [11]:
# Remove duplicates that match song name and artist name keeping the one with highest popularity
df_comb.sort_values('popularity', ascending=False, inplace=True)
df_comb.drop_duplicates(subset=['name', 'artists'], keep='first', inplace=True)

# Size check
print(len(df_comb))

24360


In [12]:
# Add column of data processing on lyrics
df_comb['clean lyric'] = df_comb['Lyric'].apply(lambda x: preprocess_text(x))

In [14]:
# Create dataframes for importing into dialogflow as raw text options
artist_list = df_comb['artists'].unique().tolist()

# Check list of genres
genre_list = df_comb['Genres'].explode().unique().tolist()
print(len(genre_list))
print(genre_list)

df_artist_list = pd.DataFrame({'name':artist_list, 'synonym':artist_list})
df_genre_list = pd.DataFrame({'name':genre_list, 'synonym':genre_list})
df_song_list = pd.DataFrame({'name':df_comb['name'], 'synonym':df_comb['name']})

79
['R&B', 'Chillout', 'Pop', 'Hip Hop', 'Rap', 'Dance', 'Romântico', 'Trap', 'K-Pop', 'K-Rock', 'Indie', 'Rock Alternativo', 'Rock', 'Soul Music', 'Trip-Hop', 'Piano Rock', 'Soft Rock', 'House', 'Electronica', 'Black Music', 'New Wave', 'Punk Rock', 'Reggaeton', 'Punk', 'Hard Rock', 'Folk', 'Trilha Sonora', 'Surf Music', 'Country', 'Forró', 'Piseiro', 'Grunge', 'Progressivo', 'Hardcore', 'Heavy Metal', 'Funk Carioca', 'Tecnopop', 'Psicodelia', 'Disco', 'Gótico', 'Pós-Punk', 'Reggae', 'Blues', 'Gospel', 'Religioso', 'Ska', 'Rockabilly', 'Tropical House', 'Post-Rock', 'Funk', 'Power-Pop', 'Jazz', 'Lo-fi', 'Industrial', 'Clássico', 'World Music', 'New Age', 'Bossa Nova', 'Instrumental', 'Emocore', 'Classic Rock', 'Pagode', 'Sertanejo', 'MPB', 'Samba', 'Trance', 'Axé', 'J-Pop', 'J-Rock', 'Infantil', 'Electro Swing', 'Jovem Guarda', 'Fado', 'Regional', 'Metal', 'Velha Guarda', 'Samba Enredo', 'Kizomba', 'Músicas Gaúchas']


Data clean-up during debug

In [20]:
# Define the string values to filter out
string_values_to_filter = ['NaN', 'nan', 'Instrumental', 'instrumental']

# Filter out rows based on the specified string values
df_comb = df_comb[~df_comb['clean lyric'].isin(string_values_to_filter)]

# Size check
print(len(df_comb))

24092


Quick sanity check on datset

In [ ]:
# Filter rows based on the 'artists' column matching the specified string
filtered_df = df_comb[df_comb['artists'] == 'louis armstrong']

# Print the filtered dataframe (confirms no more duplicates, genres correct, cleaned lyrics correct)
filtered_df

Save finalized dataframes as CSVs

In [ ]:
# Save df_comb to a CSV file
df_comb.to_csv('df_comb_v3.csv', index=False)

In [ ]:
# Save dialogflow lists to CSV files
df_artist_list.to_csv('artist_list.csv', index=False, quoting=csv.QUOTE_ALL)
df_genre_list.to_csv('genre_list.csv', index=False, quoting=csv.QUOTE_ALL)
df_song_list.to_csv('song_list.csv', index=False, quoting=csv.QUOTE_ALL)